In [65]:
import pandas as pd

In [66]:
all_post = pd.read_csv("Sentiment_Analysis_Dataset.csv", header=0, delimiter=",", error_bad_lines=False, index_col=0)

Skipping line 8836: expected 4 fields, saw 5

Skipping line 535882: expected 4 fields, saw 7



In [67]:
all_post = all_post.drop("SentimentSource", axis=1)

In [68]:
all_post = all_post.reset_index(drop=True)

In [69]:
#slice posts in two part: train and test. Get 10% for test.
test_len = int(round(all_post.shape[0] * 0.1))

In [70]:
test_len

157861

In [71]:
train_df = all_post[0: 20000]#[test_len:]#

In [72]:
train_df = train_df.reset_index(drop=True)

In [73]:
test_df = all_post[20000: 40000]#[:test_len]

In [74]:
test_df = test_df.reset_index(drop=True)

In [75]:
#x = test_df.SentimentText[9999]

In [76]:
#' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x).split())

In [108]:
emo_repl = {
    # positive emoticons
    "&lt;3": " good ",
    ":d": " good ",  # :D in lower case
    ":dd": " good ",  # :DD in lower case
    "8)": " good ",
    ":-)": " good ",
    ":)": " good ",
    ";)": " good ",
    "(-:": " good ",
    "(:": " good ",

    # negative emoticons:
    ":/": " bad ",
    ":&gt;": " sad ",
    ":')": " sad ",
    ":-(": " bad ",
    ":(": " bad ",
    ":S": " bad ",
    ":-S": " bad ",
}

emo_repl_order = [s for (lenk, s) in reversed(sorted([(len(k), k)for k in emo_repl.keys()]))]

def clean_post(raw_post):
    import HTMLParser 
    import re
    from nltk.corpus import stopwords
    from nltk.stem.snowball import SnowballStemmer
    #Download package stopwords and stemmer use:
    #import nltk
    #nltk.download()
    
    #url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    #raw_post = re.sub(url_pattern, '', raw_post, flags=re.MULTILINE)
    for k in emo_repl_order:
        raw_post = raw_post.replace(k, emo_repl[k])
    
    raw_post = HTMLParser.HTMLParser().unescape(raw_post.decode('utf-8')) #&quot - to symbol representation"
    
    letters_only = re.sub("[^a-zA-Z]", " ", raw_post)
    lower_case = letters_only.lower()
    words = lower_case.split()
    words = [word for word in words if word not in stopwords.words("english")]
    stemmer = SnowballStemmer("english")
    words = [stemmer.stem(word) for word in words]
    return " ".join(words)

In [109]:
def clean_posts(raw_posts):
    clean_posts = []
    for i in xrange(0, raw_posts.SentimentText.size):
        if (i + 1) % 10000 == 0:
            print("{0} completed".format(i + 1))
        clean_posts.append(clean_post(raw_posts.SentimentText[i]))
    return clean_posts

In [110]:
def create_bag_of_words(clean_posts):
    from sklearn.feature_extraction.text import CountVectorizer
    vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 5000) 
    data_features = vectorizer.fit_transform(clean_posts)
    #vocab = vectorizer.get_feature_names()
    return data_features.toarray()

In [111]:
clean_train_posts = clean_posts(train_df)
data_train_features = create_bag_of_words(clean_train_posts)

10000 completed
20000 completed


In [112]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators = 100) 
forest = forest.fit(data_train_features, train_df.Sentiment)

In [113]:
clean_test_posts = clean_posts(test_df)
data_test_features = create_bag_of_words(clean_test_posts)

10000 completed
20000 completed


In [114]:
result = forest.predict(data_test_features)

In [115]:
test_df['Sentiment_Result'] = result

In [116]:
test_df['Sum_Result'] = (test_df.Sentiment + test_df.Sentiment_Result) % 2

In [117]:
import numpy as np
np.sum(test_df.Sum_Result) / (np.size(test_df.Sum_Result) * 1.0)

0.5128

In [118]:
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv("Bag_of_Words_model.csv", index=False, quoting=3 )

NameError: name 'test' is not defined

In [ ]:
#import numpy as np

# Sum up the counts of each vocabulary word
#dist = np.sum(train_data_features, axis=0)
#for tag, count in zip(vocab, dist):
#    print count, tag

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
model = GaussianNB()

In [ ]:
result2= model.fit(data_train_features, train_df.Sentiment).predict(data_test_features)

In [ ]:
result2

In [23]:
timeline = pd.read_csv("timeline.csv", header=0)

In [24]:
timeline

,TweetId,TweetText
0,672190736373325824,RT @strategyzer: Train 10+ ppl from your team ...
1,672185139992989696,Make sure to check the Proposed Schedule Chang...
2,672184920660426752,Coffee &amp; Ice Cream are now being served in...
3,672180762163019776,RT @waynebeaton: Visualizing Java 9 Module Rel...
4,672177601566347264,Afternoon Sponsored Sessions are now underway!...
5,672166930267152385,List map by @Flat_Studio on @dribbble https://...
6,672164982163816448,Track every mile &amp; moment to breaking thro...
7,672164596153606144,Come join #google at the 10th #andevcon Women ...
8,672163381160226816,"RT @brunoborges: To celebrate 10,000+ follower..."
9,672159509159256064,Lunch is served in Grand Ballroom B and the Wo...
